
# Markov Chain Load Time Optimization

This notebook analyzes user behavior on a website using a Markov chain model. Each row in the dataset represents a transition from one page to another. We estimate transition probabilities, simulate load times under different preloading strategies, and analyze long-term expected performance.

---

## Step 1: Load and Prepare the Data

```python
import pandas as pd
import numpy as np

# Load data
df = pd.read_csv('data/websites.csv', header=None, names=['source', 'destination'])

# Get the number of pages
n_pages = max(df.max()) + 1  # assumes pages are labeled 0 to n-1

# Initialize count matrix
count_matrix = np.zeros((n_pages, n_pages))

# Populate count matrix
for _, row in df.iterrows():
    count_matrix[row['source'], row['destination']] += 1

# Normalize rows to get transition matrix
transition_matrix = count_matrix / count_matrix.sum(axis=1, keepdims=True)
```

---

## Step 2: Simulate Load Times from Page 1

```python
np.random.seed(42)

# Extract probabilities from page 1
probabilities = transition_matrix[1]

# Sample next pages
n_samples = 10000
next_pages = np.random.choice(n_pages, size=n_samples, p=probabilities)

# Identify top 1 and top 2 likely next pages
top1 = np.argsort(probabilities)[-1]
top2 = np.argsort(probabilities)[-2:]

# Simulate load times
load_times_1 = np.where(next_pages == top1,
                        np.random.exponential(1/10, size=n_samples),
                        np.random.exponential(1, size=n_samples))

load_times_2 = np.where(np.isin(next_pages, top2),
                        np.random.exponential(1/10, size=n_samples),
                        np.random.exponential(1, size=n_samples))
```

---

## Step 3: Compare Empirical and Theoretical Load Times

```python
avg_no_preloading = 1.0
avg_one_preload = np.mean(load_times_1)
avg_two_preload = np.mean(load_times_2)

print(f"Avg load time with no preloading: {avg_no_preloading:.4f}")
print(f"Avg load time with 1 page preloaded: {avg_one_preload:.4f}")
print(f"Avg load time with 2 pages preloaded: {avg_two_preload:.4f}")
```

> **Conclusion:** The empirical average load times are lower than the baseline of 1 second. Thus, preloading significantly improves page load times.

---

## Step 4: Stationary Distribution and Long-Term Expected Load Time

```python
# Compute stationary distribution
eigvals, eigvecs = np.linalg.eig(transition_matrix.T)
stationary_dist = np.real(eigvecs[:, np.isclose(eigvals, 1)])
stationary_dist = stationary_dist[:, 0]
stationary_dist = stationary_dist / stationary_dist.sum()

# Expected load time with one page preloaded from each state
expected_load_times = np.zeros(n_pages)
for i in range(n_pages):
    probs = transition_matrix[i]
    top = np.argmax(probs)
    expected = np.sum(
        np.where(np.arange(n_pages) == top, 1/10, 1) * probs
    )
    expected_load_times[i] = expected

# Weighted by stationary distribution
expected_load_time_stationary = np.dot(stationary_dist, expected_load_times)
print(f"Expected load time with preloading using stationary distribution: {expected_load_time_stationary:.4f}")
```

---



---

# SVD-Based Anomaly Detection

This section analyzes a dataset using Singular Value Decomposition (SVD) and detects anomalies based on reconstruction error from a low-rank approximation.

---

## Step 1: Load Data and Compute SVD

```python
import pandas as pd
import numpy as np

# Load data
df = pd.read_csv('data/SVD.csv', header=None)
problem1_data = df.values

# Perform SVD
U, s, VT = np.linalg.svd(problem1_data, full_matrices=False)
D = np.diag(s)

# First singular vectors
first_left_singular_vector = U[:, 0]
first_right_singular_vector = VT[0, :]
```

---

## Step 2: Explained Variance and Component Selection

```python
explained_variance = s**2
total_variance = np.sum(explained_variance)
explained_variance_ratio = np.cumsum(explained_variance) / total_variance

# Select number of components for 95% variance
num_components = np.searchsorted(explained_variance_ratio, 0.95) + 1
print(f"Components needed to explain 95% variance: {num_components}")
```

---

## Step 3: Best Rank-k Approximation

```python
U_k = U[:, :num_components]
D_k = D[:num_components, :num_components]
VT_k = VT[:num_components, :]
problem1_approximation = U_k @ D_k @ VT_k
```

> Each row in the approximated matrix represents a denoised or compressed version of the corresponding sample in the original data using only the most important singular directions.

---

## Step 4: Anomaly Detection Using Reconstruction Error

```python
reconstruction_errors = np.linalg.norm(problem1_data - problem1_approximation, axis=1)

# Plot ECDF
import matplotlib.pyplot as plt
sorted_errors = np.sort(reconstruction_errors)
n = len(sorted_errors)
ecdf = np.arange(1, n + 1) / n

plt.plot(sorted_errors, ecdf)
plt.xlabel("Reconstruction Error")
plt.ylabel("Empirical CDF")
plt.title("Empirical Distribution of Reconstruction Errors")
plt.grid(True)
plt.show()

# Identify top 10 anomalies
threshold = sorted_errors[-10]
anomalous_samples = np.argsort(reconstruction_errors)[-10:]
print("Indices of 10 most anomalous samples:", anomalous_samples)
```
